In [443]:
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import pandas as pd
import time
import ta 
import numpy as np
import warnings
warnings.simplefilter("ignore")

WebSocket Inverse Perp (Auth) encountered error: [WinError 10054] An existing connection was forcibly closed by the remote host.
WebSocket Inverse Perp (Auth) encountered error: [WinError 10054] An existing connection was forcibly closed by the remote host.


In [399]:
load_dotenv()

True

In [400]:
api_key_pw = os.getenv('api_key_bot')
api_secret_pw = os.getenv('api_secret_bot')


In [401]:
session = inverse_perpetual.HTTP(
    endpoint='https://api.bybit.com', 
    api_key= api_key_pw,
    api_secret= api_secret_pw
)
ws = inverse_perpetual.WebSocket(
    test=False,
    api_key= api_key_pw,
    api_secret= api_secret_pw
)

In [402]:
pd.DataFrame(session.latest_information_for_symbol(symbol ='BTCUSD')["result"]).keys()

Index(['symbol', 'bid_price', 'ask_price', 'last_price', 'last_tick_direction',
       'prev_price_24h', 'price_24h_pcnt', 'high_price_24h', 'low_price_24h',
       'prev_price_1h', 'mark_price', 'index_price', 'open_interest',
       'countdown_hour', 'turnover_24h', 'volume_24h', 'funding_rate',
       'predicted_funding_rate', 'next_funding_time',
       'predicted_delivery_price', 'total_turnover', 'total_volume',
       'delivery_fee_rate', 'delivery_time', 'price_1h_pcnt', 'open_value'],
      dtype='object')

In [403]:
session.orderbook(symbol='BTCUSD')['result'][1]

{'symbol': 'BTCUSD', 'price': '16360.5', 'side': 'Buy', 'size': 41928}

In [404]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="1m")["result"])
frame

,0,1,2,3,4,5,6,7,8,9,10
0,1669678080000,16236.68,16240.54,16236.4,16239.6,0.741756,1669678140000,12045.2983523,17,0,0
1,1669678140000,16239.6,16240.54,16237.3,16237.86,0.048427,1669678200000,786.37509474,9,0,0
2,1669678200000,16237.86,16238.57,16231.88,16234.3,2.868047,1669678260000,46561.2764776,66,0,0
3,1669678260000,16234.3,16236.14,16230,16231.4,1.694368,1669678320000,27502.7832075,43,0,0
4,1669678320000,16231.4,16236.14,16229.65,16233.98,2.303207,1669678380000,37386.84488479,45,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1669737780000,16386.48,16392.99,16380,16382.37,9.459767,1669737840000,155015.93281415,201,0,0
996,1669737840000,16382.37,16390.37,16374.62,16377.85,5.618869,1669737900000,92047.66393921,166,0,0
997,1669737900000,16377.85,16382.14,16371.75,16375.54,2.745592,1669737960000,44966.50976498,188,0,0
998,1669737960000,16375.54,16387.36,16372.43,16380.02,4.289951,1669738020000,70273.70326811,262,0,0


In [405]:
def get15minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="15m")["result"])
    frame = frame.iloc[:,: 6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [406]:
df = get15minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-11-19 06:15:00,16575.03,16619.00,16566.66,16605.67,23.576538
2022-11-19 06:30:00,16605.67,16607.24,16579.18,16597.57,18.625741
2022-11-19 06:45:00,16597.57,16610.98,16592.00,16609.00,8.662517
2022-11-19 07:00:00,16609.00,16617.00,16602.80,16613.69,19.158154
2022-11-19 07:15:00,16613.69,16616.00,16602.03,16604.67,14.135641
...,...,...,...,...,...
2022-11-29 15:00:00,16366.00,16397.76,16342.77,16368.43,83.036664
2022-11-29 15:15:00,16368.43,16420.22,16368.24,16419.44,50.196198
2022-11-29 15:30:00,16419.44,16452.32,16392.88,16394.90,82.261365


In [407]:
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14)
    #df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)


In [408]:
apply_technicals(df)

In [409]:
df.head()

,Open,High,Low,Close,Volume,K,RSI
Time,,,,,,,
2022-11-19 12:45:00,16653.25,16657.00,16638.51,16652.16,17.814216,0.088788,59.365722
2022-11-19 13:00:00,16652.16,16655.68,16639.92,16644.06,14.562126,0.000000,54.302910
2022-11-19 13:15:00,16644.06,16653.21,16632.18,16645.49,16.036503,0.038480,55.032024
2022-11-19 13:30:00,16645.49,16661.85,16642.93,16654.57,16.060336,0.271942,59.455585
2022-11-19 13:45:00,16654.57,16664.00,16647.40,16647.52,13.816177,0.033455,54.936816


In [430]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
    
    #Checking if we have a trigger in the last n time steps
    def get_trigger(self):
        df_2 = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df["RSI"].shift(i) < 20)
            df_2 = df_2.append(mask, ignore_index = True)
        return df_2.sum(axis= 0)
    
    # Is the trigger fulfilled and are all buying conditions fulfilled?
    def decide(self):
         self.df["trigger"] = np.where(self.get_trigger(), 1, 0)
         self.df["Buy"]= np.where((self.df.trigger) & (self.df.K < 0.20), 1, 0)


In [436]:
inst = Signals(df, 2)

In [437]:
inst.decide()

In [439]:
df.tail(151)

,Open,High,Low,Close,Volume,K,RSI,trigger,Buy
Time,,,,,,,,,
2022-11-28 02:30:00,16133.17,16185.45,16119.47,16143.32,73.710590,0.090769,15.194770,1,1
2022-11-28 02:45:00,16143.32,16152.94,16106.03,16119.90,56.347533,0.053972,14.225473,1,1
2022-11-28 03:00:00,16119.90,16163.90,16106.69,16157.47,46.871985,0.377203,22.739928,1,0
2022-11-28 03:15:00,16157.47,16187.58,16156.10,16178.53,44.964107,0.543024,27.107909,1,0
2022-11-28 03:30:00,16178.53,16205.00,16160.00,16202.90,33.470807,0.725151,31.905462,0,0
...,...,...,...,...,...,...,...,...,...
2022-11-29 15:00:00,16366.00,16397.76,16342.77,16368.43,83.036664,0.025562,34.563676,0,0
2022-11-29 15:15:00,16368.43,16420.22,16368.24,16419.44,50.196198,0.557688,46.137431,0,0
2022-11-29 15:30:00,16419.44,16452.32,16392.88,16394.90,82.261365,0.380674,42.264539,0,0


In [440]:
df[df.Buy==1]

,Open,High,Low,Close,Volume,K,RSI,trigger,Buy
Time,,,,,,,,,
2022-11-20 21:45:00,16436.06,16442.98,16213.62,16279.10,332.235742,0.000000,19.756722,1,1
2022-11-20 22:00:00,16279.10,16355.98,16183.73,16296.27,193.867903,0.096006,22.990057,1,1
2022-11-21 19:30:00,15814.55,15860.00,15593.48,15676.99,417.496872,0.000000,19.734500,1,1
2022-11-28 01:15:00,16388.00,16396.00,16192.95,16234.51,372.109590,0.000000,16.341904,1,1
2022-11-28 01:30:00,16234.51,16240.00,16114.32,16154.54,274.664868,0.000000,13.496908,1,1
2022-11-28 01:45:00,16154.54,16167.00,16099.47,16147.83,106.145603,0.000000,13.287875,1,1
2022-11-28 02:00:00,16147.83,16152.53,16056.02,16140.34,149.339512,0.000000,13.045008,1,1
2022-11-28 02:15:00,16140.34,16192.80,16103.35,16133.17,69.961093,0.000000,12.803755,1,1
2022-11-28 02:30:00,16133.17,16185.45,16119.47,16143.32,73.710590,0.090769,15.194770,1,1


In [441]:
def strategy(qty, open_position = False):
    df= get15minutedata()
    apply_technicals(df)
    inst = Signals(df, 5)
    inst.decide()
    print(f'Current Close is '+str(df.Close.iloc[-1]))
    if df.Buy.iloc[-1]:
        order = session_auth.place_active_order(symbol="BTCUSDT",
                                                side="Buy",
                                                type="MARKET",
                                                qty= qty,
                                                timeInForce="GTC")
        print(order)
        buyprice = float(order['result']['price'])
        open_position = True
    while open_position:
        time.sleep(2)
        df = get15minutedata()
        print(f'Current Close ' + str(df.Close.iloc[-1]))
        print(f'Potential Target ' + str(buyprice * 1.02))
        #print(f'Current Target 2 ' + str(df.RSI[-1] > 45))
        print(f'Current Stop ' + str(buyprice * 0.98))
        if df.Close[-1] <= buyprice* 0.98 or df.Close[-1] >= 1.02 * buyprice or df.RSI[-1] > 45:
            order = session_auth.place_active_order(symbol="BTCUSDT",
                                                side="Sell",
                                                type="MARKET",
                                                qty= qty,
                                                timeInForce="GTC")
            print(order)
            break

In [442]:
strategy(0.001)

Current Close is 16406.0


In [ ]:
while True: 
    strategy(0.001)
    time.sleep(1)

In [ ]:
#Get active order 
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
print(session_auth.get_active_order(
    orderId=""
))

In [ ]:
#Cancel active order
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
print(session_auth.cancel_active_order(
    orderId=""
))